# Autocorrect

This application is to understand the most commonly used feature in any smart phone, it might be while texting, web-searching, or documenting. 

#### Things to achieve autocorrect system:

   **Step 01:** Finding out the `misspelled` word.

   **Step 02:** Identify the words that are 'n' `edit-distance` away.

   **Step 03:** Eliminate the words that are not in the `corpus`.

   **Step 04:** Calculate the `probabilities` of these words.

   **Step 05:**  Identify the top three suggestions (with higher probability).

`Note: This is a simple autocorrect system that identifies the misspelled words and not the conextual meaning.`

In [2]:
import numpy as np
import pandas as pd
import re
from collections import Counter

# 1.0 Preparing the Dataset

First we need to prepare vocabulary which is extracted from the corpus given.

I am using a basic corpus with 6k+ unique words, I am going to perform a RegEx to remove the unwanted punctuations etc.

Next step, I would like to build the word_frequency dictionary  that basically has the frequency of each word.

In [7]:
def prepare_vocab():
    file = open("corpus.txt","r")
    
    file_data = (file.readline()).lower()
    words = []
    while file_data:
        words_list = re.findall(r"\w+", file_data)
        words.extend(words_list)
        file_data = (file.readline()).lower()
    word_frequency = Counter(words)   
    return set(words), word_frequency #set removes the duplicates in your list hence forming our vocabulary

In [12]:
vocabulary, word_frequency = prepare_vocab()
print(f"The number of unique words in our vocabulary {len(vocabulary)}.")
print(f"The frequency of 'the' word: {word_frequency['the']}.")

The number of unique words in our vocabulary 6116.
The frequency of 'the' word: 1525.


Now create a probability dictionary, that mean probability of each word that it occurs randomly.

In [9]:
def probability_of_words(word_frequency):
    total = sum(word_frequency.values())
    word_probability = {key: value/total for key, value in word_frequency.items()}
    return word_probability

In [13]:
word_probability = probability_of_words(word_frequency)
print(f"The number of unique words in our vocabulary {len(word_probability)}.")
print(f"The probability of 'the' word: {word_probability['the']}.")

The number of unique words in our vocabulary 6116.
The probability of 'the' word: 0.028444063117842356.


# 2.0 String Manipulations

Four main operations performed on the string to form the list of words that could be a possible replacement for the misspelled word are:

* `Delete`
* `Swap`
* `Replace`
* `Insert`

## 2.0.1 Swap_manipulation

**loot**

    [('', 'loot'), ('l', 'oot'), ('lo', 'ot'), ('loo', 't'), ('loot', '')]
    
    We will divide the given word in to a tuple at each index forming the above list

    1) First step will be to form a list of all possible tuples except the last combo.
    2) Delete each letter from the give word and hence forming a list of different words:
    
        L + R[1:], there by deleting the letter R[0]
        
        ['oot', 'lot', 'lot', 'loo']
        

In [65]:
from itertools import combinations

def delete_manipulation(word):
    word_tuples = [(word[:i],word[i:]) for i in range(len(word))] 
    return [L + R[1:] for L,R in word_tuples]

In [66]:
print(delete_manipulation("loot"))

['oot', 'lot', 'lot', 'loo']


## 2.0.2 Swap_manipulation

**loot**

    [('', 'loot'), ('l', 'oot'), ('lo', 'ot'), ('loo', 't'), ('loot', '')]
    
    We will divide the given word in to a tuple at each index forming the above list

    1) First step will be to form a list of all possible tuples except the last combo.
    2) Swap each letter with their neighboring letter forming a new string, append each string and return the list.
    
        For each tuple (L, R) we will return a new string where the string = L + swap(R[0] + R[1]) + R[2:]. 
        Resulting:
        
        ['olot', 'loot', 'loto']

In [67]:
def swap_manipulation(word):
    
    #Since the last tuple has no effect as R = "" we donot consider the range(len(R)+1)
    
    word_tuples = [(word[:i],word[i:]) for i in range(len(word))] 
    return [L+R[1]+R[0]+R[2:] for L, R in word_tuples if len(R) >= 2]

In [68]:
print(swap_manipulation("loot"))

['olot', 'loot', 'loto']


## 2.0.3 Replace_manipulations

    
    1) Prepare the list of tuples
    2) Replacing each letter with "abcdef....xyz", for each tuple (L, R) we shall return L + C + R[1:] replacing R[0] with C
    
    
['aoot', 'boot', 'coot', 'doot', 'eoot', 'foot', 'goot', 'hoot', 'ioot', 'joot', 'koot', 'laot', 'lbot', 'lcot', 'ldot', 'leot', 'lfot', 'lgot', 'lhot', 'liot', 'ljot', 'lkot', 'llot', 'lmot', 'lnot', 'loat', 'lobt', 'loct', 'lodt', 'loet', 'loft', 'logt', 'loht', 'loit', 'lojt', 'lokt', 'lolt', 'lomt', 'lont', 'looa', 'loob', 'looc', 'lood', 'looe', 'loof', 'loog', 'looh', 'looi', 'looj', 'look', 'lool', 'loom', 'loon', 'looo', 'loop', 'looq', 'loor', 'loos', 'loou', 'loov', 'loow', 'loox', 'looy', 'looz', 'lopt', 'loqt', 'lort', 'lost', 'lott', 'lout', 'lovt', 'lowt', 'loxt', 'loyt', 'lozt', 'lpot', 'lqot', 'lrot', 'lsot', 'ltot', 'luot', 'lvot', 'lwot', 'lxot', 'lyot', 'lzot', 'moot', 'noot', 'ooot', 'poot', 'qoot', 'root', 'soot', 'toot', 'uoot', 'voot', 'woot', 'xoot', 'yoot', 'zoot']

In [56]:
def replace_manipulation(word):
    """
        Here we use a seperate string for replacing each letter - alphabets
        Prepare a list of tuples that are broken at each index - word_tuples
        For every tuple (L, R) replace R[0] with 'c' in alphabets
    """
    alphabets = "abcdefghijklmnopqrstuvwxyz"
    word_tuples = [(word[:i],word[i:]) for i in range(len(word))]
    return [L + c + R[1:] for L, R in word_tuples for c in alphabets if c != R[0]]

In [57]:
print(replace_manipulation("loot"))

['aoot', 'boot', 'coot', 'doot', 'eoot', 'foot', 'goot', 'hoot', 'ioot', 'joot', 'koot', 'moot', 'noot', 'ooot', 'poot', 'qoot', 'root', 'soot', 'toot', 'uoot', 'voot', 'woot', 'xoot', 'yoot', 'zoot', 'laot', 'lbot', 'lcot', 'ldot', 'leot', 'lfot', 'lgot', 'lhot', 'liot', 'ljot', 'lkot', 'llot', 'lmot', 'lnot', 'lpot', 'lqot', 'lrot', 'lsot', 'ltot', 'luot', 'lvot', 'lwot', 'lxot', 'lyot', 'lzot', 'loat', 'lobt', 'loct', 'lodt', 'loet', 'loft', 'logt', 'loht', 'loit', 'lojt', 'lokt', 'lolt', 'lomt', 'lont', 'lopt', 'loqt', 'lort', 'lost', 'lott', 'lout', 'lovt', 'lowt', 'loxt', 'loyt', 'lozt', 'looa', 'loob', 'looc', 'lood', 'looe', 'loof', 'loog', 'looh', 'looi', 'looj', 'look', 'lool', 'loom', 'loon', 'looo', 'loop', 'looq', 'loor', 'loos', 'loou', 'loov', 'loow', 'loox', 'looy', 'looz']


## 2.0.4 Insert_manipulation

    1) Repeat the step 01 of replace, except we need the duplicate term ("word",""). Hence use range(len(word)+1)
    2) For inserting a new alphabet at every other index: L + c + R where (L, R) are tuples and c is alphabet from "abcd...xyz"
    
['aloot', 'bloot', 'cloot', 'dloot', 'eloot', 'floot', 'gloot', 'hloot', 'iloot', 'jloot', 'kloot', 'lloot', 'mloot', 'nloot', 'oloot', 'ploot', 'qloot', 'rloot', 'sloot', 'tloot', 'uloot', 'vloot', 'wloot', 'xloot', 'yloot', 'zloot', 'laoot', 'lboot', 'lcoot', 'ldoot', 'leoot', 'lfoot', 'lgoot', 'lhoot', 'lioot', 'ljoot', 'lkoot', 'lloot', 'lmoot', 'lnoot', 'looot', 'lpoot', 'lqoot', 'lroot', 'lsoot', 'ltoot', 'luoot', 'lvoot', 'lwoot', 'lxoot', 'lyoot', 'lzoot', 'loaot', 'lobot', 'locot', 'lodot', 'loeot', 'lofot', 'logot', 'lohot', 'loiot', 'lojot', 'lokot', 'lolot', 'lomot', 'lonot', 'looot', 'lopot', 'loqot', 'lorot', 'losot', 'lotot', 'louot', 'lovot', 'lowot', 'loxot', 'loyot', 'lozot', 'looat', 'loobt', 'looct', 'loodt', 'looet', 'looft', 'loogt', 'looht', 'looit', 'loojt', 'lookt', 'loolt', 'loomt', 'loont', 'looot', 'loopt', 'looqt', 'loort', 'loost', 'loott', 'loout', 'loovt', 'loowt', 'looxt', 'looyt', 'loozt', 'loota', 'lootb', 'lootc', 'lootd', 'loote', 'lootf', 'lootg', 'looth', 'looti', 'lootj', 'lootk', 'lootl', 'lootm', 'lootn', 'looto', 'lootp', 'lootq', 'lootr', 'loots', 'loott', 'lootu', 'lootv', 'lootw', 'lootx', 'looty', 'lootz']

In [58]:
def insert_manipulation(word):
    alphabets = "abcdefghijklmnopqrstuvwxyz"
    word_tuples = [(word[:i],word[i:]) for i in range(len(word)+1)] #we add the last combination ('word', '')
    return [L + c + R for L, R in word_tuples for c in alphabets]

In [59]:
print(insert_manipulation("loot"))

['aloot', 'bloot', 'cloot', 'dloot', 'eloot', 'floot', 'gloot', 'hloot', 'iloot', 'jloot', 'kloot', 'lloot', 'mloot', 'nloot', 'oloot', 'ploot', 'qloot', 'rloot', 'sloot', 'tloot', 'uloot', 'vloot', 'wloot', 'xloot', 'yloot', 'zloot', 'laoot', 'lboot', 'lcoot', 'ldoot', 'leoot', 'lfoot', 'lgoot', 'lhoot', 'lioot', 'ljoot', 'lkoot', 'lloot', 'lmoot', 'lnoot', 'looot', 'lpoot', 'lqoot', 'lroot', 'lsoot', 'ltoot', 'luoot', 'lvoot', 'lwoot', 'lxoot', 'lyoot', 'lzoot', 'loaot', 'lobot', 'locot', 'lodot', 'loeot', 'lofot', 'logot', 'lohot', 'loiot', 'lojot', 'lokot', 'lolot', 'lomot', 'lonot', 'looot', 'lopot', 'loqot', 'lorot', 'losot', 'lotot', 'louot', 'lovot', 'lowot', 'loxot', 'loyot', 'lozot', 'looat', 'loobt', 'looct', 'loodt', 'looet', 'looft', 'loogt', 'looht', 'looit', 'loojt', 'lookt', 'loolt', 'loomt', 'loont', 'looot', 'loopt', 'looqt', 'loort', 'loost', 'loott', 'loout', 'loovt', 'loowt', 'looxt', 'looyt', 'loozt', 'loota', 'lootb', 'lootc', 'lootd', 'loote', 'lootf', 'lootg', 